In [1]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Script to set ensemsble dictionary with Latin Hypercube Sampling
# Mikayla Pascual 07/22/22
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

#import 
from MatlabFuncs import *
from model import *
from loadmodel import *
from os.path import exists
import lhsmdu
import numpy as np
import pickle

In [4]:
### frontal forcings ###
#from frontalforcingsrignot import frontalforcingsrignot
#md.frontalforcings = frontalforcingsrignot()
#md.frontalforcings.subglacial_discharge = 2.6 * np.ones(md.mesh.numberofvertices)
#md.frontalforcings.thermalforcing = 15.7 * np.ones(md.mesh.numberofvertices)

In [5]:
# Load a separate model to extract the friction coefficient and rheology
md_friction = loadmodel("./Models/SAtoES_friction_coefficient.nc")
friction_coefficient = md_friction.results.StressbalanceSolution.FrictionCoefficient
rheology_B = md_friction.materials.rheology_B
md_friction = None

# Load the original model
md = loadmodel("./Models/SAtoES_inversion.nc")
md.friction.coefficient = friction_coefficient
md.materials.rheology_B = rheology_B

Opening ./Models/SAtoES_friction_coefficient.nc for reading 
group toolkits is empty
empty init
Opening ./Models/SAtoES_inversion.nc for reading 
empty init


In [6]:
### basal sliding ###
md.friction

Basal shear stress parameters: Sigma_b = coefficient^2 * Neff ^r * |u_b|^(s - 1) * u_b,
(effective stress Neff = rho_ice * g * thickness + rho_water * g * base, r = q / p and s = 1 / p)
         coefficient            : (12112,)        -- friction coefficient [SI]
         p                      : (24063,)        -- p exponent
         q                      : (24063,)        -- q exponent
         coupling               : 0               -- Coupling flag 0: uniform sheet (negative pressure ok, default), 1: ice pressure only, 2: water pressure assuming uniform sheet (no negative pressure), 3: use provided effective_pressure, 4: used coupled model (not implemented yet)
         effective_pressure     : N/A             -- Effective Pressure for the forcing if not coupled [Pa]
         effective_pressure_l...: 0               -- Neff do not allow to fall below a certain limit: effective_pressure_limit * rho_ice * g * thickness (default 0)

In [11]:
### ice temperature ###
md.materials.rheology_B

   Materials:
         rho_ice                : 917.0           -- ice density [kg/m^3]
         rho_water              : 1023.0          -- water density [kg/m^3]
         rho_freshwater         : 1000.0          -- fresh water density [kg/m^3]
         mu_water               : 0.001787        -- water viscosity [Ns/m^2]
         heatcapacity           : 2093.0          -- heat capacity [J/kg/K]
         thermalconductivity    : 2.4             -- ice thermal conductivity [W/m/K]
         temperateiceconducti...: 0.24            -- temperate ice thermal conductivity [W/m/K]
         effectiveconductivit...: 1               -- computation of effectiveconductivity: (0) arithmetic mean, (1) harmonic mean, (2) geometric mean (default)
         meltingpoint           : 273.15          -- melting point of ice at 1atm in K
         latentheat             : 334000.0        -- latent heat of fusion [J/m^3]
         beta                   : 9.8e-08         -- rate of change of melting point wit

In [2]:
### calving forcings ###
# We set the calving model (no moving front ... spclevelset is actually ignored)
from calvingvonmises import calvingvonmises
md.calving = calvingvonmises()
md.calving.stress_threshold_groundedice = md.calving.stress_threshold_groundedice * 0.5

In [10]:
# Set Latin Hypercube sampling parameters
k = lhsmdu.sample(3, 100) # Latin Hypercube Sampling with multi-dimensional uniformity, 
                        # 3 variables with 10 samples each
k = np.array(k)
k

array([[0.53037254, 0.32026347, 0.10739125, 0.6831801 , 0.407963  ,
        0.92431733, 0.59452998, 0.30512883, 0.24713031, 0.45273503,
        0.31121587, 0.06556057, 0.89326978, 0.11442491, 0.81646947,
        0.82494126, 0.13620279, 0.145442  , 0.41739796, 0.55705937,
        0.9453419 , 0.88678903, 0.2987356 , 0.42432271, 0.77412763,
        0.99764246, 0.52582554, 0.56071007, 0.65892112, 0.19030783,
        0.80748396, 0.83758794, 0.50953111, 0.12054134, 0.61596857,
        0.05427841, 0.60330633, 0.01090319, 0.93562654, 0.90479041,
        0.33104554, 0.3861249 , 0.57737953, 0.04767278, 0.49905373,
        0.09409471, 0.97953073, 0.62865429, 0.39296502, 0.35159283,
        0.96435389, 0.74401899, 0.58575719, 0.91583386, 0.43749987,
        0.1550989 , 0.73104823, 0.47858291, 0.46309765, 0.16971297,
        0.2111786 , 0.02958251, 0.84091012, 0.87231036, 0.6304658 ,
        0.95705113, 0.66850299, 0.22434071, 0.67928522, 0.75590299,
        0.20630007, 0.54845419, 0.032968  , 0.51

In [11]:
k[0] = np.interp(k[0], (k.min(), k.max()), (+1, +20))
k[1] = np.interp(k[1], (k.min(), k.max()), (-15, -1))
k[2] = np.interp(k[2], (k.min(), k.max()), (-0.4, +0.2))

In [12]:
# Create 10 ensembles with parameters from [0], [1], [2], ..., [9]
lhs_params_for_ens = {
    'basal' : k[0],
    'ice_temp' : k[1],
    'calve' : k[2]
} 
lhs_params_for_ens

# ens needs to match LHS sample number
ens = list(range(0, 100))

ens_dict = {}
for i in range(len(ens)):
    trial_name = 'A{:03d}'.format(i)
    res = dict()
    for sub in lhs_params_for_ens:
        res[sub] = lhs_params_for_ens[sub][i]
    ens_dict[trial_name] = res
ens_dict

{'A000': {'basal': 11.06461846505946,
  'ice_temp': -14.610632648514423,
  'calve': -0.14239633532102136},
 'A001': {'basal': 7.053147637341737,
  'ice_temp': -14.925851016542094,
  'calve': -0.14252347176810665},
 'A002': {'basal': 2.988921635135776,
  'ice_temp': -14.964720175353104,
  'calve': -0.13999843766960296},
 'A003': {'basal': 13.982070221507596,
  'ice_temp': -14.67567602519086,
  'calve': -0.1268239438366477},
 'A004': {'basal': 8.727535648744237,
  'ice_temp': -14.700253562140894,
  'calve': -0.1406963837952911},
 'A005': {'basal': 18.585940912033244,
  'ice_temp': -14.379298617215296,
  'calve': -0.13135048617401351},
 'A006': {'basal': 12.289533283379336,
  'ice_temp': -14.985829581277255,
  'calve': -0.14266352604560384},
 'A007': {'basal': 6.764192013370105,
  'ice_temp': -14.717999629191503,
  'calve': -0.1256995412481035},
 'A008': {'basal': 5.656865510146335,
  'ice_temp': -14.978124728348485,
  'calve': -0.13974178902280743},
 'A009': {'basal': 9.582337791557848,


In [13]:
# save output
with open('ens_dict.pickle', 'wb') as f:
    # Pickle the 'data' dictionary 
    pickle.dump(ens_dict, f)